In [1]:
import numpy as np
import time
import pandas as pd
import ast
import multiprocessing
from multiprocessing import Pool
from contextlib import closing
from functools import partial
import pickle

In [2]:
def make_all_change(string,what):
    tp = string
    tp_list = list(tp)
    loc = -1
    while 1==1:
        loc_tp = tp.find(what)
        if loc_tp==-1:
            break
        loc = loc + 1 + loc_tp
        tp_list[loc] = what[0].capitalize()
        tp = string[loc+1:]
    return "".join(tp_list)

In [3]:
def data_cleaning(features):
    clmns = ['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork', 'sessionId', 'socialEngagementType', 'totals', 'trafficSource', 'visitId', 'visitNumber', 'visitStartTime']
    train_sample = pd.DataFrame(columns=clmns)
    train_sample.loc[0] = features
    train_cleaned = pd.DataFrame()
    for cl in clmns:
        try:
            string = train_sample.loc[0,cl]

            string = make_all_change(string,'true')
            string = make_all_change(string,'false')

            temp_dict = ast.literal_eval(string)

            for key in temp_dict.keys():
                if type(temp_dict[key]) is dict:
                    temp_dict_2 = temp_dict[key]
                    for keys_n in temp_dict_2.keys():
                        if type(temp_dict_2[keys_n]) is dict:
                            temp_dict_3 = temp_dict_2[keys_n]
                            for keys_n_n in temp_dict_3.keys():
                                train_cleaned.loc[0, keys_n_n] = temp_dict_3[keys_n_n]
                        else:
                            train_cleaned.loc[0,keys_n] = temp_dict_2[keys_n]
                else:
                    train_cleaned.loc[0,key] = temp_dict[key]
        except:
            train_cleaned.loc[0,cl]=train_sample.loc[0,cl]
        with open('col_names.pickle', 'rb') as pkl:
            col_vect = pickle.load(pkl)
        for key in col_vect.keys():
            if key not in train_cleaned.columns:
                train_cleaned[key] = np.NaN
    return train_cleaned.values[0]

In [4]:
path_train = "train.csv"
path_test = "test.csv"

t = time.time()
train_set = pd.read_csv(path_train, sep=',', header=0)
print(time.time() - t)

t = time.time()
train_set_list = []
train_count = len(train_set)
for i in range(train_count):
    train_set_list.append(train_set.loc[i].tolist())
print(time.time() - t)

t = time.time()
with closing(Pool(multiprocessing.cpu_count())) as pool:
    cleaned_train_list = pool.map(data_cleaning, train_set_list)
print(time.time() - t)

/home/vibudh/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/rdkit-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


17.169654607772827
184.59459400177002
1846.990475177765


In [6]:
cleaned_train_array = np.array(cleaned_train_list)

In [79]:
with open('col_names.pickle', 'rb') as pkl:
        col_vect = pickle.load(pkl)
t = time.time()
print(cleaned_train_array.shape)
with open('trained_cleaned.csv','w') as ref:
    ref.write(',;'.join(list(col_vect.keys())))
    ref.write('\n')
    for i in range(train_count):
        try:
            ref.write(',;'.join(str(elem) for elem in cleaned_train_array[i]))
            ref.write('\n')
        except:
            ref.write(',;'.join(str(elem) for elem in cleaned_train_array[i]).encode('utf-16', 'surrogatepass').decode('utf-16'))
            ref.write('\n')
            print(i)
print(time.time() - t)

(903653, 55)
564203
20.193065881729126


In [80]:
clean_train = pd.read_csv('trained_cleaned.csv',sep=',;',header=0)        

/home/vibudh/.local/easybuild/software/2017/Core/miniconda3/4.3.27/envs/rdkit-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [81]:
clean_train

,channelGrouping,date,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,...,isTrueDirect,referralPath,page,slot,gclId,adNetworkType,isVideoAd,adContent,transactionRevenue,campaignCode
0,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902.0,Firefox,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902.0,UC Browser,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Organic Search,20160902.0,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Organic Search,20160902.0,Internet Explorer,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Organic Search,20160902.0,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
"sdf,qr\n".rstrip().rsplit(',')

['sdf', 'qr']